In [1]:
import pandas as pd
import numpy as np
import os
import re
import time

In [2]:
# 合并所有解析出来的经纬度数据文件
def get_filenames(pattern):

    # 目标目录路径，根据实际情况替换，比如 "/soil_data/result/"
    target_dir = "./"
    # pattern = r'^[MRW].*(csv)$' 
    result = []
    # 遍历目录下的文件
    for filename in os.listdir(target_dir):
        if re.match(pattern, filename):
            result.append(filename)
            
    return(result)

In [3]:
Water_input_reference1 = "2018年水输入总量(汇总对照表).csv"
Water_input_reference2 = "2019年水输入总量(汇总对照表).csv"
df1 = pd.read_csv(Water_input_reference1)
df2 = pd.read_csv(Water_input_reference2)

In [4]:
pattern = r'^[MRW].*(csv)$'
filenames = get_filenames(pattern)

In [5]:
result_df = pd.DataFrame()
for filename in filenames:

    # 记录开始时间
    start_time = time.time()  # 返回当前时间的时间戳（秒）
            
    # 读取数据
    df3 = pd.read_csv(filename)
    
    water_inputs = []
    for k in range(len(df1)):
        
        start = df3.loc[df3['grid_num'] == df1.loc[k, 'grid_num'], 'start_month'].iloc[0]
        end = df3.loc[df3['grid_num'] == df1.loc[k, 'grid_num'], 'end_month'].iloc[0]
    
        if np.isfinite(start) and np.isfinite(end):
            
            start = int(start)
            end = int(end)
    
            if start<=end:
                water_inputs.append(sum(df1.values[k,start:end+1]))
            else:
                water_inputs.append(sum(df1.values[k,start:13])+sum(df2.values[k,1:end+1])) #出现跨年情形
        else:
            water_inputs.append(np.nan)
    
    file_df = pd.DataFrame({'grid_num':df1['grid_num'], f'water_input_{filename.rsplit('(',1)[0]}':water_inputs})
    # 合并数据：首次循环直接赋值，后续按经纬度关联合并
    if result_df.empty:
        result_df = file_df
    else:
        # 按经纬度匹配合并，确保同一经纬度的月份数据对应
        result_df = result_df.merge(file_df, on='grid_num', how='outer')

    # 记录结束时间
    end_time = time.time()
    # 计算耗时
    elapsed_time = end_time - start_time
    print(f"耗时: {elapsed_time/60:.2f} 分")
    
result_df.round(2).to_csv('2018年各种农作物水输入总量(汇总).csv', index=False)

耗时: 3.62 分
耗时: 0.98 分
耗时: 3.12 分
耗时: 3.29 分
耗时: 3.43 分
耗时: 1.00 分


In [6]:
result_df[result_df[result_df.columns[1:]].values>0]

,grid_num,water_input_Wheat.2,water_input_Rice.2,water_input_Wheat,water_input_Maize,water_input_Rice,water_input_Maize.2
49184,49185,573.6,NaN,NaN,343.4,369.4,NaN
49184,49185,573.6,NaN,NaN,343.4,369.4,NaN
49184,49185,573.6,NaN,NaN,343.4,369.4,NaN
49185,49186,576.3,NaN,427.0,350.3,373.6,NaN
49185,49186,576.3,NaN,427.0,350.3,373.6,NaN
...,...,...,...,...,...,...,...
249429,249430,23.0,NaN,NaN,NaN,31.0,NaN
250133,250134,7.0,NaN,NaN,NaN,14.0,NaN
250133,250134,7.0,NaN,NaN,NaN,14.0,NaN
250134,250135,8.0,NaN,NaN,NaN,15.0,NaN
